# Modelová úloha - vzdálenosti mezi body v 3D
Zadání: pro N bodů v 3 rozměrném prostoru spočítejte vzájemnou vzdálenost $d$, která je pro dva body $x,y$ definovaná jako $\sqrt {\sum_{i=1}^3 {{{\left( {{x_i} - {y_i}} \right)}^2}} } $. Výslekem je tedy (symetrická) matice $N\times N$.

### Vstupní data
Začneme čistě v Pythonu, body reprezentujeme jako list touplů.

In [ ]:
# vygeneruje list bodů, tuplů (x, y, z)
import random
n = 200
points = [(random.random(), random.random(), random.random()) for i in range(n)]

In [ ]:
# vykreslí body
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter([p[0] for p in points], [p[1] for p in points], [p[2] for p in points])
plt.show()

### Prvnotní nástřel - vzdálenost mezi body

In [ ]:
import math

def dist_py1(points):
    distances_all = []
    for point_1 in points:
        distances = []
        for point_2 in points:
            tmp_sum = 0
            for k in range(3):
                rozdil = point_1[k] - point_2[k]
                tmp_sum += rozdil ** 2
            tmp_dist = math.sqrt(tmp_sum)
            distances.append(tmp_dist)
        distances_all.append(distances)

    return distances_all


In [ ]:
%time res1 = dist_py1(points)

In [ ]:
plt.imshow(res1)

### Profilování

In [ ]:
%prun res1 = dist_py1(points)

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -u 1e-6 -f dist_py1 res1 = dist_py1(points)

#### Scalene
Mnoho profilerů potřebuje pro plnou funkcionalitu být spuštěno z příkazové řádky (jako zde Scalene).

Proto uložíme náš kód do souboru, včetně generování vstupních dat a volání funkce, kterou chceme profilovat.

In [ ]:
%%writefile test.py
import math

def dist_py1(points):
    distances_all = []
    for point_1 in points:
        distances = []
        for point_2 in points:
            tmp_sum = 0
            for k in range(3):
                rozdil = point_1[k] - point_2[k]
                tmp_sum += rozdil ** 2
            tmp_dist = math.sqrt(tmp_sum)
            distances.append(tmp_dist)
        distances_all.append(distances)

    return distances_all

import random
n = 1000
points = [(random.random(), random.random(), random.random()) for i in range(n)]
res = dist_py1(points)

In [ ]:
!scalene test.py

In [ ]:
!pprofile test.py

## Optimalizace na úrovni algoritmu

Často je možné získat větší rychlost bez nutnosti optimalizace na úrovni kódu, ale lepším algoritmem. Mezi standardní příklady patří třídící algoritmy, kde různé algoritmy mají různou složitost. 

V našem případě si můžeme všimnout, že výsledek je symetrický, tedy pro všechny body platí, že vzdálenost od bodu $A$ k bodu $B$ je stejná jako vzdálenost od bodu $B$ k bodu $A$, navíc na diagonále je vždy nula.

Dále můžeme nahradit `x**2` za `x*x`, které bude pravděpodobně ryhlejší.

In [ ]:
def dist_py2(points):
    # nejdříve vyrobíme prázdný 2d list
    distances_all: list[list[float]] = [[0 for _ in range(len(points))] 
                                        for _ in range(len(points))]
    for i in range(len(points)):
        for j in range(i+1,len(points)): # vyplníme jen hodnoty nad diagonálou
            tmp_sum = 0
            for k in range(3):
                rozdil = points[i][k] - points[j][k]
                tmp_sum += rozdil * rozdil
            tmp_dist = math.sqrt(tmp_sum)
            distances_all[i][j] = tmp_dist
            distances_all[j][i] = tmp_dist

    return distances_all

In [ ]:
%time res2 = dist_py2(points)

In [ ]:
# pro srovnání
%time res1 = dist_py1(points)

Raději zkontrolujeme, že počítáme stále to stejné:

In [ ]:
# zkontrolujeme jestli res1 a res2 jsou stejné pomocí numpy allclose
import numpy as np
print(np.allclose(res1, res2))

Podíváme se na profilování nové verze:

In [ ]:
%lprun -u 1e-6 -f dist_py2 res2 = dist_py2(points)

In [ ]:
# pro srovnání
%lprun -u 1e-6 -f dist_py1 res1 = dist_py1(points)

## Optimalizace pomocí vektorizace (NumPy)

Nejprve přepíšeme funkci tak, aby pracovala s NumPy polemi:

In [ ]:
import numpy as np
import math
def dist_np1(points):
    n = points.shape[0]
    distances_all = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            tmp_sum = 0
            for k in range(3):
                rozdil = points[i, k] - points[j, k]
                tmp_sum += rozdil*rozdil
            tmp_dist = math.sqrt(tmp_sum)
            distances_all[i, j] = tmp_dist
            distances_all[j, i] = tmp_dist

    return distances_all

In [ ]:
points_np = np.array(points)

In [ ]:
%time res3 = dist_np1(points_np)

Tohle vůbec nepomohlo, ba naopak!

Důvodem je fakt, že `numpy` je rozhraní do implementace v nízkoúrovňovém jazyce C/Fortran, které se však zde volá stále dokola (v cyklech), a tedy nasbíráme obrovský overhead.

In [ ]:
# pro jistotu overíme, že výsledky jsou stejné
print(np.allclose(res1, res3))

## Vektorizace - zbavme se vnitřních smyček

- pokusíme se zbavit vnitřních cyklů

In [ ]:
import numpy as np
import math
def dist_np2(points):
    n = points.shape[0]
    distances_all = np.zeros((n, n))
    for i in range(n):
        rozdily = points[i, :] - points[i+1: n, :]
        tmp_sum = np.sum(rozdily * rozdily, axis=1)
        tmp_dist = np.sqrt(tmp_sum)
        distances_all[i, i+1: n] = tmp_dist
        distances_all[i+1: n, i] = tmp_dist

    return distances_all

In [ ]:
%time res4 = dist_np2(points_np)

To už je je mnohem lepší! 

Raději zkontrolujeme, že počítáme stále to stejné:

In [ ]:
# kontrola
print(np.allclose(res1, res4))

In [ ]:
# jak vypadá profilování?
%lprun -u 1e-6 -f dist_np2 res4 = dist_np2(points_np)

## Úplná vektorizace na úkor redundance výpočtů

Vrátíme se zpět k původnímu kódu ale zkusíme vektorizovat vše, tedy ať nepoužíváme žádné cykly.

In [ ]:
import numpy as np
import math
def dist_np3(points):
    # Použijeme broadcasting a spočítáme všechny rozdíly najednou
    diffs = points[:, None, :] - points # shape (n, n, 3)
    
    # spočteme druhou mocninu všech rozdílů a sečteme skrz poslední osu
    tmp_sum = np.sum(diffs*diffs, axis=-1)

    distances_all = np.sqrt(tmp_sum)
    return distances_all

In [ ]:
%time res5 = dist_np3(points_np)

In [ ]:
# kontrola
print(np.allclose(res1, res5))

Zdá se, že poslední redundantní verze je nejrychlejší!

Teď je nařadě benchmarkování. Skutečně je to nejrychlejší vždy?

In [ ]:
import time
import matplotlib.pyplot as plt

velikosti =[2**i for i in range(4, 12)]
time_py1 = []
time_py2 = []
time_np2 = []
time_np3 = []

for n in velikosti:
    points = [(random.random(), random.random(), random.random()) for i in range(n)]
    points_np = np.array(points)
    start = time.time()
    res1 = dist_py1(points)
    time_py1.append(time.time() - start)
    start = time.time()
    res2 = dist_py2(points)
    time_py2.append(time.time() - start)
    start = time.time()
    res3 = dist_np2(points_np)
    time_np2.append(time.time() - start)
    start = time.time()
    res4 = dist_np3(points_np)
    time_np3.append(time.time() - start)

    print(n, time_py1[-1], time_py2[-1], time_np2[-1], time_np3[-1])

# log-log grafy
plt.loglog(velikosti, time_py1, label="py1")
plt.loglog(velikosti, time_py2, label="py2")
plt.loglog(velikosti, time_np2, label="np2")
plt.loglog(velikosti, time_np3, label="np3")
plt.legend()

## Jak si povede Numba?

Nejlepší byl zatím kód `dist_np2`, zkusíme jej "zakomplilovat" pomocí Numby.

In [ ]:
from numba import jit
dist_np2_numba = jit(dist_np2, nopython=True)

In [ ]:
# srovnání dist_np2 a dist_np2_numba
points_np = np.random.rand(1000, 3)
res1 = dist_np2(points_np)
res2 = dist_np2_numba(points_np)
print(np.allclose(res1, res2))

Při testování se kód zároveň kompiluje což je třeba mít na mysli při měření času.

In [ ]:
%timeit res1 = dist_np2(points_np)

In [ ]:
%timeit res2 = dist_np2_numba(points_np)

Zkusíme ještě původní verzi se smyčkami `dist_np1`:

In [ ]:
dist_np1_numba = jit(dist_np1, nopython=True)

In [ ]:
res1 = dist_np2(points_np)
res2 = dist_np1_numba(points_np)
print(np.allclose(res1, res2))

In [ ]:
%timeit res2 = dist_np1_numba(points_np)

Obojí výrazně zrychlilo běh, a kupodivu `dist_np1_numba` je ještě rychlejší než `dist_np2_numba`!

Toto je důsledek toho, že Numba si sama překládá kód do C a optimalizuje. Užití vektorizace jí vnucuje jeden konktrétní způsob, který nemusí být vždy nejlepší (jako je to zde).

Přidáme ji do benchmarku:

In [ ]:
import time
import matplotlib.pyplot as plt

velikosti =[2**i for i in range(6, 14)]
time_np2 = []
time_np1_numba = []
time_np2_numba = []

for n in velikosti:
    points = [(random.random(), random.random(), random.random()) for i in range(n)]
    points_np = np.array(points)
    start = time.time()
    _ = dist_np2(points_np)
    time_np2.append(time.time() - start)
    start = time.time()
    _ = dist_np1_numba(points_np)
    time_np1_numba.append(time.time() - start)
    start = time.time()
    _ = dist_np2_numba(points_np)
    time_np2_numba.append(time.time() - start)

    print(n, time_np2[-1], time_np1_numba[-1], time_np2_numba[-1])

# log-log grafy
plt.loglog(velikosti, time_np2, label="np2")
plt.loglog(velikosti, time_np1_numba, label="np1_numba")
plt.loglog(velikosti, time_np2_numba, label="np2_numba")
plt.legend()

## A co Cython?

In [ ]:
%load_ext Cython

In [ ]:
%%cython

import numpy as np
cimport numpy as np
from libc.math cimport sqrt
cimport cython

ctypedef np.float64_t DTYPE_t

@cython.boundscheck(False)
@cython.wraparound(False)
def dist_cython(np.ndarray[DTYPE_t, ndim=2] points):
    cdef int n = points.shape[0]
    cdef np.ndarray[DTYPE_t, ndim=2] distances_all = np.zeros((n, n), dtype=np.float64)
    cdef int i, j, k
    cdef double tmp_sum, rozdil, tmp_dist

    for i in range(n):
        for j in range(i+1, n):
            tmp_sum = 0
            for k in range(3):
                rozdil = points[i, k] - points[j, k]
                tmp_sum += rozdil * rozdil
            tmp_dist = sqrt(tmp_sum)
            distances_all[i, j] = tmp_dist
            distances_all[j, i] = tmp_dist

    return distances_all


In [ ]:
# vyzkoušíme si, jestli to funguje
points_np = np.random.rand(1000, 3)
res5 = dist_np2(points_np)
res6 = dist_cython(points_np)
np.allclose(res5, res6)

In [ ]:
# časové měření
%timeit res6 = dist_cython(points_np)

In [ ]:
# porovnání s numba
%timeit res5 = dist_np1_numba(points_np)

## Porovnání nejlepších variant

In [ ]:
import time
import matplotlib.pyplot as plt

velikosti =[2**i for i in range(6, 14)]
time_np2 = []
time_np1_numba = []
time_cython = []

for n in velikosti:
    points_np = np.random.rand(n, 3)
    start = time.time()
    _ = dist_np2(points_np)
    time_np2.append(time.time() - start)
    start = time.time()
    _ = dist_np1_numba(points_np)
    time_np1_numba.append(time.time() - start)
    start = time.time()
    _ = dist_cython(points_np)
    time_cython.append(time.time() - start)

    print(n, time_np2[-1], time_np1_numba[-1], time_cython[-1])

# log-log grafy
plt.loglog(velikosti, time_np2, label="np2")
plt.loglog(velikosti, time_np1_numba, label="np1_umba")
plt.loglog(velikosti, time_cython, label="cython")
plt.legend()

# Modelová úloha - hledání kam se dostaneme v grafu pokud můžeme projít n hran

### Vstupní data - list dvojic vrcholů = hrany

In [ ]:
import random
n = 10

def vygeneruj_graf(n):
    V = [i for i in range(n)]
    E = []
    start_V = 0
    end_V = 0
    while len(E) < 2*n:
        start_V = end_V
        end_V = random.randint(0, n - 1)
        E.append((start_V, end_V))
    return V, E

V, E = vygeneruj_graf(n)

In [ ]:
print(V)
print(E)

## Ochutnávka knihovny networkx = vykreslení grafu

In [ ]:
# !pip install networkx

In [ ]:
# plot graph with vertices V and edges E
# showing vertices with numbers and connections as lines
import matplotlib.pyplot as plt
import networkx as nx

G = nx.Graph()
G.add_nodes_from(V)
G.add_edges_from(E)
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos)
nx.draw_networkx_edges(G, pos)
nx.draw_networkx_labels(G, pos)
plt.show()


## První návrh - jednoduchý Python, použití setů, listů a union

In [ ]:
def reachable_in_n_steps(edges, n):
    reachable = set()
    reachable.add(0)
    for i in range(n):
        new_reachable = set()
        for v in reachable:
            for e in edges:
                if e[0] == v:
                    new_reachable.add(e[1])
                if e[1] == v:
                    new_reachable.add(e[0])
        reachable = reachable.union(new_reachable)
    return list(reachable)


In [ ]:
reachable_in_n_steps(E, 2)

## Vygenerujeme větší graf

In [ ]:
V, E = vygeneruj_graf(1000)

Jak dlouho to asi potrvá?

In [ ]:
%time res1 = reachable_in_n_steps(E, 20)

## Profilování

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f reachable_in_n_steps reachable_in_n_steps(E, 20)

Edge procházíme desítky miliónů krát... tohle ani rychlé být nemůže. Jak to můžeme zrychlit?

- edge který jednou projdeme už nemusíme znovu procházet
- druhý if v cyklu může být v elif, protože pokud je splněn první if, tak druhý už nás stejně nezajímá

## Optimalizace algoritmu

In [ ]:
from copy import deepcopy

def reachable_in_n_steps_v2(edges, n):
    edges_copy = deepcopy(edges)
    reachable = set()
    reachable.add(0)
    for i in range(n):
        new_reachable = set()
        for v in reachable:
            edges_to_remove = []
            for e_idx, e in enumerate(edges_copy):
                if e[0] == v:
                    new_reachable.add(e[1])
                    edges_to_remove.append(e_idx)
                elif e[1] == v:
                    new_reachable.add(e[0])
                    edges_to_remove.append(e_idx)
            _ = [edges_copy.pop(e_idx) for e_idx in edges_to_remove[::-1]]
        reachable = reachable.union(new_reachable)
    return list(reachable)


In [ ]:
%time res2 = reachable_in_n_steps_v2(E, 20)

Tohle bylo vskutku výrazné zrychlení!

Raději ověříme, že počítáme stále to samé:

In [ ]:
import numpy as np
np.allclose(np.array(res1), np.array(res2))

## První optimalizace - použití NumPy a pole bool hodnot místo setů

Budeme chtít také vstup jako Numpy:

In [ ]:
E_np = np.array(E)

In [ ]:
def reachable_in_n_steps_np(edges, n):
    edges_copy = edges.copy()
    n_vertices = np.max(edges_copy) + 1
    reachable = np.zeros((n_vertices), dtype=np.bool8)
    reachable[0] = True
    for i in range(n):
        new_reachable = np.zeros((n_vertices), dtype=np.bool8)
        for v, is_reachable in enumerate(reachable):
            if not is_reachable:
                continue
            edges_hits = np.logical_or(edges_copy[:,0] == v, edges_copy[:,1] == v)
            if np.sum(edges_hits) == 0:
                continue
            reachable_vertices = edges_copy[edges_hits,:].ravel()
            new_reachable[reachable_vertices] = True
            edges_copy = edges_copy[~edges_hits,:]
        reachable = np.logical_or(reachable, new_reachable)
        
    return np.where(reachable)

In [ ]:
%time res3 = reachable_in_n_steps_np(E_np, 20)

In [ ]:
np.allclose(np.array(res1), np.array(res3))

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f reachable_in_n_steps_np reachable_in_n_steps_np(E_np, 20)

In [ ]:
from numba import jit
import numpy as np

@jit(nopython=True)
def reachable_in_n_steps_numba(edges, n):
    edges_copy = edges.copy()
    n_vertices = np.max(edges_copy) + 1
    reachable = np.zeros((n_vertices), dtype=np.bool8)
    reachable[0] = True
    for i in range(n):
        new_reachable = np.zeros((n_vertices), dtype=np.bool8)
        for v, is_reachable in enumerate(reachable):
            if not is_reachable:
                continue
            edges_hits = np.logical_or(edges_copy[:,0] == v, edges_copy[:,1] == v)
            if np.sum(edges_hits) == 0:
                continue
            reachable_vertices = edges_copy[edges_hits,:].ravel()
            new_reachable[reachable_vertices] = True
            edges_copy = edges_copy[~edges_hits,:]
        reachable = np.logical_or(reachable, new_reachable)
        
    return np.where(reachable)

In [ ]:
%time res4 = reachable_in_n_steps_numba(E_np, 20)

To už je obrovské zrychlení!

Zkontrolujeme, že počítáme stále to samé:

In [ ]:
np.allclose(np.array(res1), np.array(res4))

## Zkusíme to úplně jinak - použití matice sousednosti

In [ ]:
from scipy.sparse import csc_matrix

def reachable_in_n_steps_scipy(edges, n):
    n_vertices = np.max(edges) + 1
    idx_row = np.concatenate((edges[:,0], edges[:,1]))
    idx_col = np.concatenate((edges[:,1], edges[:,0]))
    values = np.ones((len(idx_row)), dtype=np.bool_)
   
    adjacence_csc = csc_matrix((values, (idx_row, idx_col)), 
                               shape=(n_vertices, n_vertices), dtype=np.bool_)

    reachable = np.zeros((n_vertices), dtype=np.bool_)
    reachable[0] = True
    for i in range(n):
        reachable = adjacence_csc.dot(reachable)

    return np.where(reachable)[0]

In [ ]:
%time res5 = reachable_in_n_steps_scipy(E_np, 20)

To je rychlé!

Zkontrolujeme, že počítáme stále to samé:

In [ ]:
np.allclose(np.array(res1), res5)

In [ ]:
# profilovani
%lprun -f reachable_in_n_steps_scipy reachable_in_n_steps_scipy(E_np, 20)

## Benchmarkování nejlepších variant

### Rostoucí n

In [ ]:
import time
num_vert = 2000
V, E = vygeneruj_graf(num_vert)
E_np = np.array(E)

n_list = [2**i for i in range(0, 9)]
times_v2 = [] # reachable_in_n_steps_v2
times_np = [] # reachable_in_n_steps_np
times_numba = [] # reachable_in_n_steps_numba
times_scipy = [] # reachable_in_n_steps_scipy

for n in n_list:
    start = time.time()
    res = reachable_in_n_steps_v2(E, n)
    end = time.time()
    times_v2.append(end - start)
    
    start = time.time()
    res = reachable_in_n_steps_np(E_np, n)
    end = time.time()
    times_np.append(end - start)
    
    start = time.time()
    res = reachable_in_n_steps_numba(E_np, n)
    end = time.time()
    times_numba.append(end - start)
    
    start = time.time()
    res = reachable_in_n_steps_scipy(E_np, n)
    end = time.time()
    times_scipy.append(end - start)

    print(n, times_v2[-1], times_np[-1], times_numba[-1], times_scipy[-1])

# plot logaritmic scale on y axis
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.loglog(n_list, times_v2, label='v2')
plt.loglog(n_list, times_np, label='np')
plt.loglog(n_list, times_numba, label='numba')
plt.loglog(n_list, times_scipy, label='scipy')
plt.legend()

### Rostoucí pořet vrcholů

In [ ]:
import time
n = 200
num_vert_list = [2**i for i in range(5, 14)]
times_v2 = [] # reachable_in_n_steps_v2
times_np = [] # reachable_in_n_steps_np
times_numba = [] # reachable_in_n_steps_numba
times_scipy = [] # reachable_in_n_steps_scipy

for num_vert in num_vert_list:
    V, E = vygeneruj_graf(num_vert)
    E_np = np.array(E) 

    start = time.time()
    res = reachable_in_n_steps_v2(E, n)
    end = time.time()
    times_v2.append(end - start)
    
    start = time.time()
    res = reachable_in_n_steps_np(E_np, n)
    end = time.time()
    times_np.append(end - start)
    
    start = time.time()
    res = reachable_in_n_steps_numba(E_np, n)
    end = time.time()
    times_numba.append(end - start)
    
    start = time.time()
    res = reachable_in_n_steps_scipy(E_np, n)
    end = time.time()
    times_scipy.append(end - start)

    print(num_vert, times_v2[-1], times_np[-1], times_numba[-1], times_scipy[-1])

# plot logaritmic scale on y axis
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.loglog(num_vert_list, times_v2, label='v2')
plt.loglog(num_vert_list, times_np, label='np')
plt.loglog(num_vert_list, times_numba, label='numba')
plt.loglog(num_vert_list, times_scipy, label='scipy')
plt.legend()